<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.1: Your First Chisel Module
**Prev: [Introduction to Scala](1_intro_to_scala.ipynb)**<br>
**Next: [Combinational Logic](2.2_comb_logic.ipynb)**

## Motivation
既然你已经熟悉了Scala，现在让我们开始构建一些硬件吧！Chisel代表**C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage。这意味着它是一个基于Scala的领域特定语言（DSL），允许你在同一代码中利用Scala和Chisel编程。理解哪些代码是“Scala”，哪些代码是“Chisel”很重要，但我们稍后会详细讨论这一点。现在，请将Chisel和第2模块中的代码视为编写Verilog的更好方式。本模块向你介绍了整个Chisel `Module` 和测试器。现在只需了解它的大意即可。你稍后会看到更多的示例。

## Setup
以下单元格下载 Chisel 所需的依赖项。你将在所有未来的笔记本中看到它。**现在运行此单元格**。

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

如上一模块所述，这些语句是导入 Chisel 所需的。**现在运行此单元格**，然后再运行任何后续代码块。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

---
# Your First Module
本节将介绍你的第一个硬件模块、一个测试用例，以及如何运行它。这将包含许多你暂时不理解的内容，这没关系。我们希望你能够掌握整体思路，这样你可以不断回到这个完整且可运行的示例，以巩固你所学到的知识。

<span style="color:blue">**Example: A Module**</span><br>
像 Verilog 一样，我们可以在 Chisel 中声明模块定义。以下示例是一个 Chisel `Module`，`Passthrough`，它有一个 4 位输入，`in`，和一个 4 位输出，`out`。该模块在组合逻辑上连接 `in` 和 `out`，因此 `in` 驱动 `out`。

In [ ]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

内容很多！以下解释了如何从我们描述的硬件角度理解每一行代码。

```scala
class Passthrough extends Module {
```
我们声明了一个名为 `Passthrough` 的新模块。`Module` 是所有硬件模块必须继承的 Chisel 内置类。

```scala 
val io = IO(...)
```
我们在一个特殊的 `io` `val` 中声明所有的输入和输出端口。它必须叫 `io` 并且是一个 `IO` 对象或实例，这需要形式为 `IO(_instantiated_bundle_)` 的内容。

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
我们声明一个新的硬件结构类型（Bundle），它包含一些命名信号 `in` 和 `out`，方向分别是 Input 和 Output。

```scala
UInt(4.W)
```
我们声明一个信号的硬件类型。在这种情况下，它是一个宽度为4的无符号整数。

```scala
io.out := io.in
```
我们将输入端口连接到输出端口，使得 `io.in` 驱动 `io.out`。注意，`:=` 操作符是 ***Chisel*** 操作符，表示右侧信号驱动左侧信号。它是一个有方向的操作符。

硬件构建语言（HCL）的巧妙之处在于，我们可以将底层编程语言用作脚本语言。例如，在声明了我们的 Chisel 模块之后，我们使用 Scala 调用 Chisel 编译器将 Chisel `Passthrough` 转换为 Verilog `Passthrough`。这个过程称为 ***elaboration***。

In [ ]:
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

<span style="color:blue">**Example: A Module Generator**</span><br>
如果我们将学到的Scala知识应用到这个例子中，我们可以看到一个Chisel模块是作为一个Scala类实现的。就像其他Scala类一样，我们可以让一个Chisel模块接受一些构造参数。在这种情况下，我们创建一个新的类 `PassthroughGenerator`，它将接受一个整数 `width`，用于指定其输入和输出端口的宽度：

In [ ]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// Let's now generate modules with different widths
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

请注意，生成的 Verilog 根据分配给 `width` 参数的值，为输入/输出使用不同的位宽。让我们深入了解它的工作原理。由于 Chisel 模块是普通的 Scala 类，我们可以利用 Scala 类构造函数的强大功能来参数化我们设计的展开过程。

你可能会注意到，这种参数化是由*Scala*启用的，而不是*Chisel*；Chisel 没有额外的 API 用于参数化，但设计者可以简单地利用 Scala 的特性来参数化他的设计。

因为 `PassthroughGenerator` 不再描述单个模块，而是描述了由 `width` 参数化的一系列模块，我们称这个 `Passthrough` 为 ***generator***。

---
# Testing Your Hardware

没有测试器的硬件模块或生成器是不完整的。Chisel 具有内置的测试功能，你将在整个训练营中探索这些功能。以下示例是一个 Chisel 测试架，它将值传递给 `Passthrough` 实例的输入端口 `in`，并检查在输出端口 `out` 上是否看到相同的值。

<span style="color:blue">**Example: A Tester**</span><br>
这里有一些高级的 Scala 代码。然而，你只需要理解 `poke` 和 `expect` 命令即可。你可以将其余代码视为编写这些简单测试的样板代码。

In [ ]:
// Scala Code: `test` runs the unit test. 
// test takes a user Module and has a code block that applies pokes and expects to the 
// circuit under test (c)
test(new Passthrough()) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


发生了什么？测试接受一个 `Passthrough` 模块，给模块的输入赋值，并检查其输出。要设置输入，我们调用 `poke`。要检查输出，我们调用 `expect`。如果我们不想将输出与预期值进行比较（不进行断言），可以改为 `peek` 输出。

如果所有 `expect` 语句都为真，那么我们的样板代码将返回通过。

>注意，`poke` 和 `expect` 使用的是 Chisel 硬件字面值表示法。两个操作都期望提供正确类型的字面值。
如果 `poke` 一个 `UInt()`，你必须提供一个 `UInt` 字面值（例如：`c.io.in.poke(10.U)`），同样如果输入是 `Bool()`，则 `poke` 需要提供 `true.B` 或 `false.B`。

<span style="color:red">**Exercise: Writing Your Own Testers**</span><br>
编写并执行两个测试，一个测试 `PassthroughGenerator` 的宽度为 10，另一个测试宽度为 20。对于每个测试至少检查两个值：零和指定宽度支持的最大值。注意，三个问号在 Scala 中有特殊含义。你可能会在这些训练营练习中频繁看到它。运行包含 `???` 的代码将产生 `NotImplementedError`。用你自己的代码替换 `???`。

In [ ]:
// Test with width 10

test(???) { c =>
    ???
}

// Test with width 20

test(???) { c =>
    ???
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying it)</label>
<article>
<pre style="background-color:#f7f7f7">
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1023.U)
    c.io.out.expect(1023.U)
}

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1048575.U)
    c.io.out.expect(1048575.U)
}

</pre></article></div></section></div>

---
# Looking at Generated Verilog/FIRRTL

如果你在理解生成的硬件时遇到困难，并且熟悉阅读结构化的 Verilog 和/或 FIRRTL（Chisel 的中间表示，它相当于 Verilog 的仅用于综合的子集），那么你可以尝试查看生成的 Verilog，以了解 Chisel 执行的结果。

这里是一个生成 Verilog（你已经见过）和 FIRRTL 的示例。

In [ ]:
// Viewing the Verilog for debugging
println(getVerilog(new Passthrough))

In [ ]:
// Viewing the firrtl for debugging
println(getFirrtl(new Passthrough))

---
# You're done!

[Return to the top.](#top)

## <span style="color:red"> Appendix: A Note on "printf" Debugging</span>
[使用打印语句调试](https://stackoverflow.com/a/189570) 并不是总是最好的调试方法，但通常是了解问题出在哪里的一个简单的第一步。
由于 Chisel generators 是生成硬件的程序，所以打印 generator 和电路状态时有一些额外的细微差别。
记住打印语句何时执行以及打印的内容非常重要。
你可能想要打印的三种常见场景有一些重要的区别：
* Chisel generator 在电路生成期间打印
* 电路在电路仿真期间打印
* 测试器在测试期间打印

`println` 是一个内置的 Scala 函数，打印到控制台。它 **不能** 用于电路仿真期间的打印，因为生成的电路是 FIRRTL 或 Verilog，而不是 Scala。

以下代码块显示了不同风格的打印。

In [ ]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("Print during simulation: Input is %d\n", io.in)
    // chisel printf has its own string interpolator too
    printf(p"Print during simulation: IO is $io\n")

    println(s"Print during generation: Input is ${io.in}")
}

test(new PrintingModule ) { c =>
    c.io.in.poke(3.U)
    c.clock.step(5) // circuit will print
    
    println(s"Print during testing: Input is ${c.io.in.peek()}")
}